In [113]:
import numpy as np
import json
from datetime import datetime, timedelta

In [171]:
def get_urgency_priorities(limits):
    urgencies=[]
    for limit in limits:
        level_key=([key for key in limit.keys() if 'level' in key.lower()]+[None])[0]
        if level_key is not None:
            limit_levels=limit[level_key]
            for x in limit_levels:
                x['upperlower']=x.get('Upper') or x.get('upper') or -x.get('Lower') or -x.get('lower')
            limit_levels=sorted([x for x in limit_levels if x['upperlower'] is not None],key=lambda x: x['upperlower'])
            
            for limitlevel in limit_levels:
                cur_urgency=limitlevel.get('Urgency') or limitlevel.get('urgency')
                if cur_urgency is not None and cur_urgency.lower() not in urgencies:
                    urgencies.append(cur_urgency.lower())
    return urgencies

def get_equipment_limits(equipment_id,limits,property_name):
    levels=dict(upper=[],lower=[],urgency=[])
    for limit in limits:
        limit_equipment=limit.get('Equipment') or limit.get('equipment')
        if type(limit_equipment) not in [list,tuple,np.ndarray]:
            limit_equipment=[limit_equipment]
        if limit_equipment is None or equipment_id in limit_equipment:
            prop = limit.get('Property') or limit.get('property')
            level_key=([key for key in limit.keys() if 'level' in key.lower()]+[None])[0]
            if prop is not None and level_key is not None and prop.lower()==property_name.lower():
                for limitlevel in limit[level_key]:
                    levels['upper'].append(limitlevel.get('Upper') or limitlevel.get('upper') or np.inf)
                    levels['lower'].append(limitlevel.get('Lower') or limitlevel.get('lower') or -np.inf)
                    levels['urgency'].append(limitlevel.get('Urgency') or limitlevel.get('urgency'))
    return {key:np.array(vals) for key,vals in levels.items()}

def predict(config,samples,limits):
    #Loop through samples
    cur_t=0;cur_urgency=None;cur_sample=None
    conditions=[]
    ordered_urgencies=get_urgency_priorities(limits)
    conditions=[]
    for sample in samples:
        sample_equipment=sample.get('Equipment') or sample.get('equipment')
        if type(sample_equipment) not in [list,tuple,np.ndarray]:
            sample_equipment=[sample_equipment]
        data = sample.get('Data') or sample.get('data')

        if data:
            for property_name, values in data.items():
                t,y=zip(*[(x['t'],x['y']) for x in sorted(values,key=lambda x: x['t'])])
                y=np.array(y)
                t=np.array(t)

                for equipment_id in sample_equipment:
                    equipment_limits = get_equipment_limits(equipment_id,limits,property_name)
                    y_upper=y-equipment_limits['upper'].reshape((-1,1))
                    y_lower=equipment_limits['lower'].reshape((-1,1))-y
                    y_all=np.vstack([y_upper,y_lower])
                    y_all=np.where(np.logical_or(np.isnan(y_all),y_all<0),np.inf,y_all)
                    pos=~np.all(np.isinf(y_all),axis=0)
                    ind=np.argmin(y_all,axis=0)
                    urgency=np.tile(equipment_limits['urgency'],2)
                    cur_conditions=np.where(pos,urgency[ind],None)
                    
                    prev_cond=None
                    for t,condition in zip(t[pos],urgency[ind][pos]):
                        if prev_cond is None or condition!=prev_cond:
                            conditions.append({
                                'Equipment_id':equipment_id,
                                'Property':property_name,
                                'Sample':sample['_id'],
                                'Condition':condition,
                                'Timestamp':t,
                                'Version':config.VERSION,
                                'Extra':(config.EXTRA_RETURN_ARGS if config.EXTRA_RETURN_ARGS else {}),
                            })
                            prev_cond=condition
    #Clean up
    conditions=sorted(conditions,key=lambda x: x['Timestamp'])
    res=[]
    for condition in conditions:
        prev_cond=[x for x in res if x['Timestamp']<=condition['Timestamp'] and x['Equipment_id']<=condition['Equipment_id']]
        if len(prev_cond)==0 or (condition['Property']==prev_cond[-1]['Property'] and condition['Condition']!=prev_cond[-1]['Condition']) or (condition['Property']!=prev_cond[-1]['Property'] and ordered_urgencies.index(condition['Condition'].lower())>ordered_urgencies.index(prev_cond[-1]['Condition'].lower())):
            res.append(condition)
    return res

In [173]:
pdays=5
days=20
s_per_day=60*60*24
equipment=[1]

samples=[]
limits=[]
for sample_id,eqid in enumerate(equipment):
    
    tsnow=datetime.timestamp(datetime.now())

    t=np.linspace(tsnow-(s_per_day*days),tsnow,s_per_day)
    t=np.arange(days)
    counts=np.cumsum(np.random.poisson(10,days))
    t_p=np.linspace(t[0],t[-1],pdays)#-(s_per_day/2)
    p=np.random.random(pdays)*100
    samples.append({
        "_id":sample_id,
        "Sensor":5,
        "Equipment":eqid,
        "Data":{
            'Counts':[{'t':t,'y':y} for t,y in zip(t,counts)],
            'Power':[{'t':t,'y':y} for t,y in zip(t_p,p)],
        }
    })
    limits.append({
        "_id":sample_id,
        "Equipment":eqid,
        "Property":"Counts",
        "Admin":True,
        "LimitLevel":[
            {
              "_id":1,
              "Limit":1,
              "Lower":None,
              "Upper":50,
              "Urgency":"Low Warning"
            },
            {
              "_id":2,
              "Limit":1,
              "Lower":None,
              "Upper":150,
              "Urgency":"Low Alarm"
            }]
    })
    limits.append({
        "_id":sample_id+1,
        "Equipment":eqid,
        "Property":"Power",
        "Admin":True,
        "LimitLevel":[
            {
              "_id":1,
              "Limit":1,
              "Lower":10,
              "Upper":50,
              "Urgency":"Low Warning"
            },
            {
              "_id":2,
              "Limit":1,
              "Lower":None,
              "Upper":80,
              "Urgency":"Low Alarm"
            }]
    })
samples,limits

([{'_id': 0,
   'Sensor': 5,
   'Equipment': 1,
   'Data': {'Counts': [{'t': 0, 'y': 12},
     {'t': 1, 'y': 23},
     {'t': 2, 'y': 38},
     {'t': 3, 'y': 54},
     {'t': 4, 'y': 62},
     {'t': 5, 'y': 74},
     {'t': 6, 'y': 82},
     {'t': 7, 'y': 90},
     {'t': 8, 'y': 100},
     {'t': 9, 'y': 110},
     {'t': 10, 'y': 122},
     {'t': 11, 'y': 130},
     {'t': 12, 'y': 140},
     {'t': 13, 'y': 149},
     {'t': 14, 'y': 160},
     {'t': 15, 'y': 175},
     {'t': 16, 'y': 183},
     {'t': 17, 'y': 194},
     {'t': 18, 'y': 205},
     {'t': 19, 'y': 211}],
    'Power': [{'t': 0.0, 'y': 13.463096296890843},
     {'t': 4.75, 'y': 60.63104533160135},
     {'t': 9.5, 'y': 86.03547228449744},
     {'t': 14.25, 'y': 54.2183568231845},
     {'t': 19.0, 'y': 24.743291066523433}]}}],
 [{'_id': 0,
   'Equipment': 1,
   'Property': 'Counts',
   'Admin': True,
   'LimitLevel': [{'_id': 1,
     'Limit': 1,
     'Lower': None,
     'Upper': 50,
     'Urgency': 'Low Warning'},
    {'_id': 2,
  

In [157]:
get_urgency_priorities(limits)

['low warning', 'low alarm']

In [175]:
predict(config,samples,limits)

[{'Equipment_id': 1,
  'Property': 'Counts',
  'Sample': 0,
  'Condition': 'Low Warning',
  'Timestamp': 3,
  'Version': 1,
  'Extra': {}},
 {'Equipment_id': 1,
  'Property': 'Power',
  'Sample': 0,
  'Condition': 'Low Alarm',
  'Timestamp': 9.5,
  'Version': 1,
  'Extra': {}},
 {'Equipment_id': 1,
  'Property': 'Power',
  'Sample': 0,
  'Condition': 'Low Warning',
  'Timestamp': 14.25,
  'Version': 1,
  'Extra': {}}]

In [152]:
y>equipment_limits['upper'].reshape((-1,1))

array([[False,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False,  True,  True,  True]])

In [74]:
y_upper=y-equipment_limits['upper'].reshape((-1,1))
y_lower=equipment_limits['lower'].reshape((-1,1))-y
y_all=np.vstack([y_upper,y_lower])
y_all=np.where(y_all<0,np.inf,y_all)
pos=~np.all(np.isinf(y_all),axis=0)
pos,np.argmin(y_all,axis=0)
ind=np.argmin(y_all,axis=0)
urgency=np.tile(equipment_limits['urgency'],2)
urgency[ind]

array(['Low Warning', 'Low Warning', 'Low Warning', 'Low Warning',
       'Low Warning', 'Low Alarm', 'Low Alarm', 'Low Alarm'], dtype='<U11')

In [50]:
equipment_limits['lower'].reshape((-1,1))-y

array([[   5.,  -43.,  -52.,  -76.,  -85.,  -95.,  -97., -110.],
       [ -55., -103., -112., -136., -145., -155., -157., -170.]])